In [ ]:
! pip install -U git+https://github.com/IINemo/isanlp.git@discourse

In [ ]:
SERVER0 = ''
SERVER1 = ''
SERVER2 = ''

In [2]:
from isanlp.processor_remote import ProcessorRemote
from isanlp import PipelineCommon
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

address_morph = (SERVER0, 4333)
address_syntax = (SERVER0, 5336)
address_rst = (SERVER2, 3343)


ppl = PipelineCommon([(ProcessorRemote(address_morph[0], address_morph[1], 'default'),
                 ['text'],
                 {'tokens': 'tokens',
                  'sentences': 'sentences',
                  'postag': 'mystem_postag',
                  'lemma': 'lemma'}),
                (ProcessorRemote(address_syntax[0], address_syntax[1], '0'),
                 ['tokens', 'sentences'],
                 {'syntax_dep_tree': 'syntax_dep_tree'}),
                (ConverterMystemToUd(),
                 ['mystem_postag'],
                 {'morph': 'morph',
                  'postag': 'postag'}),
                (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
                 ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
                 {'rst': 'rst'})
                     ])

In [3]:
%%time

text = "Мир изменился. Евросоюзу стало сложнее пропагандировать идею свободного пересечения границ. Идею, на которой основываются остальные ценности ЕС."
result = ppl(text)

CPU times: user 8.84 ms, sys: 3.44 ms, total: 12.3 ms
Wall time: 951 ms


In [4]:
result['rst']

In [5]:
print(result['rst'][0].right)

id: 3
text: Евросоюзу стало сложнее пропагандировать идею свободного пересечения границ. Идею, на которой основываются остальные ценности ЕС.
relation: contrast_m
left: Евросоюзу стало сложнее пропагандировать идею свободного пересечения границ.
right: Идею, на которой основываются остальные ценности ЕС.
start: 15
end: 145


In [6]:
def extr_pairs(tree):
    pp = []
    
    if tree.left:
        pp.append([tree.left.text, tree.right.text, tree.relation])
        pp += extr_pairs(tree.left)
        pp += extr_pairs(tree.right)
    return pp

In [7]:
extr_pairs(result['rst'][0])

[['Мир изменился.',
  'Евросоюзу стало сложнее пропагандировать идею свободного пересечения границ. Идею, на которой основываются остальные ценности ЕС.',
  'root'],
 ['Евросоюзу стало сложнее пропагандировать идею свободного пересечения границ.',
  'Идею, на которой основываются остальные ценности ЕС.',
  'contrast_m']]